### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import locale as locale
import numpy as np

In [2]:
locale.setlocale(locale.LC_ALL, '')
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data = purchase_data.set_index('Purchase ID')
#test if the data is good
#print(len(purchase_data.dropna( )) == len(purchase_data))
#purchase_data['Gender'].value_counts( )

In [3]:
#locale.localeconv( )

In [4]:

m = round(purchase_data['Price'].mean( ))
#print(locale.currency(m))

In [5]:
#purchase_data.head()


## Player Count

* Display the total number of players


In [6]:
tp = len(purchase_data['SN'].unique( ))
pd.DataFrame({'Total Players': [ tp ] })


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [7]:

n_items = len( purchase_data['Item Name'].unique( ) )
n_purchases =  len(purchase_data)
n_revenue = purchase_data['Price'].sum( )
avg_sale = purchase_data['Price'].mean( )

purch_summary_df = pd.DataFrame( {
    'Number of Unique Items':[ n_items ],
    'Average Price':[ locale.currency( avg_sale ) ],
    'Number of Purchases':[ n_purchases ],
    'Total Revenue':[ locale.currency( n_revenue , grouping = True) ]
})

purch_summary_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
#did anyone change gender?


id_gb = purchase_data.groupby(['SN'])
id_gender_gb = purchase_data.groupby(['SN','Gender'])
#if len(id_gender_gb) == len(id_gb):
    #print("All cis")
#else:
    #print("Trans Lives Matter")
    
#or have a birthday?
id_age_gb = purchase_data.groupby(['SN','Age'])
#if len(id_age_gb) == len(id_gb):
    #print("I'll stop time")
#else:
    #print("Happy birthday to " + str(len(id_age_gb) - len(id_gb)) + " players!")

In [9]:
#since our game is too new for any players to have changed their settings, we can collapse the user data:


In [10]:
total_sales_by_sn = id_gb.sum( )['Price']
total_purchase_by_user = pd.DataFrame(total_sales_by_sn).rename( columns={'Price': 'Total Purchase'})
#total_purchase_by_user

In [11]:
#total_sales_by_sn[id_gb.count()['Gender'] > 1]
#this does not count the sales by trans users

In [12]:
#total_sales_by_sn[id_gb.count()['Gender'] == 1].sum()

In [13]:
#id_gb.count()['Gender'].value_counts()

In [14]:
#player_gb = purchase_data.groupby(['SN','Gender','Age'])['Gender']
#player_gb.count()

In [15]:
user_data = purchase_data.groupby(['SN']).first( )
user_data = user_data[['Gender','Age']]
user_data['Purchase Count'] = id_gb.count( )['Age']
user_data['Total Purchases'] = total_purchase_by_user['Total Purchase']
#user_data.head( )

In [16]:
gender_gb = purchase_data.groupby(['Gender'])
gender_total_sales = gender_gb['Price'].sum( )
gender_total_sales = pd.DataFrame(gender_total_sales)
gender_total_sales = gender_total_sales.rename( columns={'Price': 'Total Purchase Value'})
#gender_total_sales 

In [17]:
sale_count_by_gender = gender_gb.count( )['SN']
sale_count_by_gender = pd.DataFrame(sale_count_by_gender)
sale_count_by_gender = sale_count_by_gender.rename( columns={'SN':'Purchase Count'})
#sale_count_by_gender

In [18]:
users_by_gender = user_data.groupby('Gender').count( ).rename( columns={'Age': 'Number of Users'})
users_by_gender['Percentage of Players'] = users_by_gender['Number of Users']/tp*100
users_by_gender['Percentage of Players'] = users_by_gender['Percentage of Players'].map('{:.2f}%'.format)
users_by_gender[['Number of Users','Percentage of Players']]

,Number of Users,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [19]:
average_sale_by_gender = gender_total_sales['Total Purchase Value'] / sale_count_by_gender['Purchase Count']
#average_sale_by_gender

In [20]:
gender_summary = sale_count_by_gender
gender_summary['Average Purchase Price'] = average_sale_by_gender.map(locale.currency)

gender_summary = pd.merge(gender_summary,gender_total_sales,on='Gender')

gender_summary['Avg Total Purchase per Person'] = (
    gender_total_sales['Total Purchase Value'] / users_by_gender['Number of Users']
).map(locale.currency)

gender_summary['Total Purchase Value'] = gender_summary['Total Purchase Value'].map(
    lambda c: locale.currency(c, grouping=True)
)

gender_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [21]:
pd.DataFrame(user_data['Age'].describe().map('{:.2f}'.format))

,Age
count,576.00
mean,22.74
std,6.84
min,7.00
25%,19.00
50%,22.00
75%,25.00
max,45.00


In [22]:
bins = [0,10,14,19,24,29,34,39,1000]
#print(len(bins))
group_names = ['<10','10-14','15-19','20-24','24-29','29-34','35-39','40+']
#print(len(group_names))

In [23]:
user_data['Cohort'] = pd.cut(user_data['Age'],bins,labels=group_names,include_lowest=True)
#user_data

In [24]:
age_summary = pd.DataFrame(user_data.groupby('Cohort').count()['Age'])
age_summary = age_summary.rename(columns={'Age':'Total Count'})
age_summary['Percent'] = ( age_summary['Total Count'] / tp * 100 ).map("{:,.2f}%".format)

age_summary 

,Total Count,Percent
Cohort,,
<10,24,4.17%
10-14,15,2.60%
15-19,107,18.58%
20-24,258,44.79%
24-29,77,13.37%
29-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [25]:
purchase_data['Cohort'] = pd.cut(purchase_data['Age'],bins,labels=group_names,include_lowest=True)

age_summary['Total Purchase Value'] = purchase_data.groupby('Cohort').sum()['Price']
age_summary['Purchase Count'] = purchase_data.groupby('Cohort').count()['Price']

#age_summary

In [26]:
age_summary['Average Purchase Price'] = age_summary['Total Purchase Value'] / age_summary['Purchase Count']
age_summary['Avg Total Purchase per Person'] = age_summary['Total Purchase Value'] / age_summary['Total Count']

#age_summary

In [27]:
for col in ['Avg Total Purchase per Person','Total Purchase Value','Average Purchase Price']:
    age_summary[col]=age_summary[col].map(lambda x: locale.currency(x,grouping=True))
    


In [28]:
age_summary_2 = age_summary[['Purchase Count','Average Purchase Price','Total Purchase Value','Avg Total Purchase per Person']]
age_summary_2

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Cohort,,,,
<10,32,$3.40,$108.96,$4.54
10-14,19,$2.68,$50.95,$3.40
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
24-29,101,$2.90,$293.00,$3.81
29-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [39]:

user_data['Average Purchase Price'] = user_data['Total Purchases']/user_data['Purchase Count']

user_data

top_user = user_data.sort_values(by='Total Purchases',ascending=False)[['Purchase Count',
                                                                          'Average Purchase Price',
                                                                          'Total Purchases']]
for col in ['Average Purchase Price','Total Purchases']:
    top_user[col]=top_user[col].map(lambda x: locale.currency(x,grouping=True))
top_user = top_user.rename(columns={'Total Purchases':'Total Purchase Value'})
top_user.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [30]:

item_gb = purchase_data[[ 'Item ID','Item Name','Price' ]].groupby(['Item ID','Item Name'])
item_summary = pd.DataFrame(item_gb.count()['Price'])
item_summary = item_summary.rename(columns={'Price':'Purchase Count'})

#item_summary

In [31]:
item_summary['Item Price'] = item_gb.first()['Price']

In [32]:
item_summary['Total Purchase Value'] = item_gb.sum()['Price']
item_summary = item_summary.sort_values(by='Purchase Count',ascending=False)


In [33]:
item_pp = item_summary.copy()
for col in ['Item Price','Total Purchase Value']:
    item_pp[col]=item_pp[col].map(lambda x: locale.currency(x,grouping=True))
item_pp.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [34]:
item_summary = item_summary.sort_values(by='Total Purchase Value',ascending=False)
#item_summary.head()

In [35]:
item_pp = item_summary.copy()
for col in ['Item Price','Total Purchase Value']:
    item_pp[col]=item_pp[col].map(lambda x: locale.currency(x,grouping=True))
item_pp.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


In [36]:
#You must include a written description of three observable trends based on the data.

In [41]:
__analysis__ = [
    "Our players are mostly men, aged 20-24.",
    "Our most popular and profitable item is the Final Critic.",
    "Most players (who made a purchase) only buy 1 or 2 items."
]

for insight in __analysis__:
    print(insight)

Our players are mostly men, aged 20-24.
Our most popular and profitable item is the Final Critic.
Most players (who made a purchase) only buy 1 or 2 items.
